In [ ]:
# Open in Colab and click the link
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
! git clone https://github.com/youzhanghe123/Lora.git
%cd Lora

fatal: destination path 'Lora' already exists and is not an empty directory.
/content/Lora


In [ ]:
import sys
sys.path.append('.')  # This makes 'src' importable

In [ ]:
!ls -Lora
!ls -Lora src/

total 280
drwxr-xr-x 3 root   4096 Nov 10 20:59 src
-rw-r--r-- 1 root    436 Nov 10 20:57 run.sh
-rw-r--r-- 1 root     40 Nov 10 20:57 requirements.txt
-rw-r--r-- 1 root    535 Nov 10 20:57 README.md
-rw-r--r-- 1 root   1082 Nov 10 20:57 main.py
-rw-r--r-- 1 root 252015 Nov 10 20:57 Lora_experiment.ipynb
drwxr-xr-x 8 root   4096 Nov 10 20:57 .git
drwxr-xr-x 1 root   4096 Nov 10 20:57 ..
drwxr-xr-x 4 root   4096 Nov 10 20:57 .
total 52
-rw-r--r-- 1 root  6566 Nov 10 20:57 trainer.py
drwxr-xr-x 2 root  4096 Nov 10 20:59 __pycache__
-rw-r--r-- 1 root  9529 Nov 10 20:57 lora.py
-rw-r--r-- 1 root 14318 Nov 10 20:57 loranew.py
-rw-r--r-- 1 root  3853 Nov 10 20:57 dataset.py
drwxr-xr-x 4 root  4096 Nov 10 20:57 ..
drwxr-xr-x 3 root  4096 Nov 10 20:59 .


In [ ]:
!pip3 install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.9/208.9 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:


In [ ]:
import datasets

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Demo of loading saved lora state_dict

In [ ]:
# Read the weights file
# Path to your .pt file
weights_path = "/content/drive/MyDrive/lora_outputs/final_lora_weights.pt"

def analyze_lora_weights(weights_path: str):
    """
    Perform detailed analysis of LoRA weights
    """
    checkpoint = torch.load(weights_path, map_location='cpu')

    if 'lora_state' not in checkpoint:
        print("No LoRA state found in checkpoint")
        return
    print(checkpoint.keys())
    lora_state = checkpoint['lora_state']

    print(lora_state.keys())

    # Analyze weight statistics
    print("\n=== Weight Statistics ===")



    return lora_state



In [ ]:
lora_state = analyze_lora_weights(weights_path)

<ipython-input-9-c4904b83f598>:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(weights_path, map_location='cpu')


dict_keys(['lora_state', 'final_perplexity'])
odict_keys(['model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'model.decoder.final_layer_norm.weight', 'model.decoder.final_layer_norm.bias', 'model.decoder.layers.0.self_attn.k_proj.lora_A', 'model.decoder.layers.0.self_attn.k_proj.lora_B', 'model.decoder.layers.0.self_attn.k_proj.existing_layer.weight', 'model.decoder.layers.0.self_attn.k_proj.existing_layer.bias', 'model.decoder.layers.0.self_attn.v_proj.lora_A', 'model.decoder.layers.0.self_attn.v_proj.lora_B', 'model.decoder.layers.0.self_attn.v_proj.existing_layer.weight', 'model.decoder.layers.0.self_attn.v_proj.existing_layer.bias', 'model.decoder.layers.0.self_attn.q_proj.lora_A', 'model.decoder.layers.0.self_attn.q_proj.lora_B', 'model.decoder.layers.0.self_attn.q_proj.existing_layer.weight', 'model.decoder.layers.0.self_attn.q_proj.existing_layer.bias', 'model.decoder.layers.0.self_attn.out_proj.weight', 'model.decoder.layers.0.self_attn.out_proj.bias',

In [ ]:
lora_state.keys()

odict_keys(['model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'model.decoder.final_layer_norm.weight', 'model.decoder.final_layer_norm.bias', 'model.decoder.layers.0.self_attn.k_proj.lora_A', 'model.decoder.layers.0.self_attn.k_proj.lora_B', 'model.decoder.layers.0.self_attn.k_proj.existing_layer.weight', 'model.decoder.layers.0.self_attn.k_proj.existing_layer.bias', 'model.decoder.layers.0.self_attn.v_proj.lora_A', 'model.decoder.layers.0.self_attn.v_proj.lora_B', 'model.decoder.layers.0.self_attn.v_proj.existing_layer.weight', 'model.decoder.layers.0.self_attn.v_proj.existing_layer.bias', 'model.decoder.layers.0.self_attn.q_proj.lora_A', 'model.decoder.layers.0.self_attn.q_proj.lora_B', 'model.decoder.layers.0.self_attn.q_proj.existing_layer.weight', 'model.decoder.layers.0.self_attn.q_proj.existing_layer.bias', 'model.decoder.layers.0.self_attn.out_proj.weight', 'model.decoder.layers.0.self_attn.out_proj.bias', 'model.decoder.layers.0.self_attn_layer_norm.

In [ ]:
type(lora_state["model.decoder.layers.0.self_attn.k_proj.lora_A"])

torch.Tensor

In [ ]:
lora_state["model.decoder.layers.0.self_attn.k_proj.lora_A"].shape

torch.Size([8, 768])

In [ ]:
lora_state["model.decoder.layers.0.self_attn.k_proj.lora_B"].shape

torch.Size([768, 8])

In [ ]:
lora_state["model.decoder.layers.0.self_attn.k_proj.existing_layer.weight"].shape

torch.Size([768, 768])

# Finetune LLama2

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
# Use the end-of-sequence (eos) token as the padding token
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm):

In [ ]:
# All-in-one training script
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from src.loranew import add_lora_to_model
from src.dataset import load_and_prepare_data
from src.trainer import LoRATrainer
import os

# Setup
output_dir = "/content/drive/MyDrive/lora_outputs"
os.makedirs(output_dir, exist_ok=True)

# Add LoRA
model, _ = add_lora_to_model(
    model,

    target_modules=[
        "0.self_attn.q_proj",  # Match q_proj in any self_attn
        "*.up_proj"              # Match any FFN layers
    ],
    auto_detect=False,

    save_path = output_dir
)
print("yes model lora")

Matched module: model.layers.0.self_attn.q_proj for target: 0.self_attn.q_proj
Matched module: model.layers.0.mlp.up_proj for target: *.up_proj
Matched module: model.layers.1.mlp.up_proj for target: *.up_proj
Matched module: model.layers.2.mlp.up_proj for target: *.up_proj
Matched module: model.layers.3.mlp.up_proj for target: *.up_proj
Matched module: model.layers.4.mlp.up_proj for target: *.up_proj
Matched module: model.layers.5.mlp.up_proj for target: *.up_proj
Matched module: model.layers.6.mlp.up_proj for target: *.up_proj
Matched module: model.layers.7.mlp.up_proj for target: *.up_proj
Matched module: model.layers.8.mlp.up_proj for target: *.up_proj
Matched module: model.layers.9.mlp.up_proj for target: *.up_proj
Matched module: model.layers.10.self_attn.q_proj for target: 0.self_attn.q_proj
Matched module: model.layers.10.mlp.up_proj for target: *.up_proj
Matched module: model.layers.11.mlp.up_proj for target: *.up_proj
Matched module: model.layers.12.mlp.up_proj for target: *.u

In [ ]:
# Prepare data
train_dataloader, test_encodings = load_and_prepare_data(
    tokenizer,
    batch_size=64,
    chunk_size=256,
    subset_fraction=0.1  # Use 0.1 for testing, 1.0 for full training
)
print("yes data lora")
# Train
trainer = LoRATrainer(
    model=model,
    train_dataloader=train_dataloader,
    test_encodings=test_encodings,
    save_dir=output_dir,
    device = device
)


Loading train split of WikiText...


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Created 5752 chunks
Loading test split of WikiText...
Created 6853 chunks
yes data lora


In [ ]:
trainer.train()
print("yes train lora")
# Save
save_path = os.path.join(output_dir, 'final_lora_weights.pt')
torch.save({
    'lora_state': model.state_dict(),
    'final_perplexity': trainer.best_perplexity
}, save_path)

Training:  14%|█▍        | 13/90 [1:03:03<6:14:02, 291.46s/it, loss=23.4]

# Finetune visiual transformer

In [ ]:
from src.loranew import add_lora_to_model
from src.dataset import load_and_prepare_data
from src.trainer import LoRATrainer
import os

In [ ]:
from transformers import ViltForQuestionAnswering

model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

In [ ]:
model

ViltForQuestionAnswering(
  (vilt): ViltModel(
    (embeddings): ViltEmbeddings(
      (text_embeddings): TextEmbeddings(
        (word_embeddings): Embedding(30522, 768)
        (position_embeddings): Embedding(40, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (patch_embeddings): ViltPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
      )
      (token_type_embeddings): Embedding(2, 768)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViltEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViltLayer(
          (attention): ViltAttention(
            (attention): ViltSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=76

In [ ]:
output_dir = "/content/drive/MyDrive/lora_outputs_vit"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
model, _ = add_lora_to_model( model,

target_modules=[
    "*.attention.query",  # Match q_proj in any self_attn
    "*.intermediate.dense"     # Match any FFN layers
],
auto_detect=False,

save_path = output_dir
)

Matched module: vilt.encoder.layer.0.attention.attention.query for target: *.attention.query
Matched module: vilt.encoder.layer.0.intermediate.dense for target: *.intermediate.dense
Matched module: vilt.encoder.layer.1.attention.attention.query for target: *.attention.query
Matched module: vilt.encoder.layer.1.intermediate.dense for target: *.intermediate.dense
Matched module: vilt.encoder.layer.2.attention.attention.query for target: *.attention.query
Matched module: vilt.encoder.layer.2.intermediate.dense for target: *.intermediate.dense
Matched module: vilt.encoder.layer.3.attention.attention.query for target: *.attention.query
Matched module: vilt.encoder.layer.3.intermediate.dense for target: *.intermediate.dense
Matched module: vilt.encoder.layer.4.attention.attention.query for target: *.attention.query
Matched module: vilt.encoder.layer.4.intermediate.dense for target: *.intermediate.dense
Matched module: vilt.encoder.layer.5.attention.attention.query for target: *.attention.que

In [ ]:
model

ViltForQuestionAnswering(
  (vilt): ViltModel(
    (embeddings): ViltEmbeddings(
      (text_embeddings): TextEmbeddings(
        (word_embeddings): Embedding(30522, 768)
        (position_embeddings): Embedding(40, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (patch_embeddings): ViltPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
      )
      (token_type_embeddings): Embedding(2, 768)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViltEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViltLayer(
          (attention): ViltAttention(
            (attention): ViltSelfAttention(
              (query): LoRAAdapter(
                (lora_dropout): Dropout(p=0.1, inplace=False)
                (existing_layer): LoRAAdapter(
                  (lora_dropout): Dropout(p=0.1, 

In [ ]:
model.to(device)

ViltForQuestionAnswering(
  (vilt): ViltModel(
    (embeddings): ViltEmbeddings(
      (text_embeddings): TextEmbeddings(
        (word_embeddings): Embedding(30522, 768)
        (position_embeddings): Embedding(40, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (patch_embeddings): ViltPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32))
      )
      (token_type_embeddings): Embedding(2, 768)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViltEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViltLayer(
          (attention): ViltAttention(
            (attention): ViltSelfAttention(
              (query): LoRAAdapter(
                (lora_dropout): Dropout(p=0.1, inplace=False)
                (existing_layer): Linear(in_features=768, out_features=768, bias=True)
         

In [ ]:
from transformers import ViltProcessor

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
